In [96]:
# Load libraries
import pandas as pd
import re
import random
from pygments.token import Other

In [97]:
# Load dataset
url = "https://raw.githubusercontent.com/maddawg9838/K-means-Algorithm/refs/heads/main/usnewshealth.txt"

df = pd.read_csv(url, sep="|", header=None, names=["tweet_id", "timestamp", "text"])

In [98]:
df.head()

,tweet_id,timestamp,text
0,586278450392133633,Thu Apr 09 21:24:09 +0000 2015,Planning to hire a personal trainer? Read thes...
1,586260156155043843,Thu Apr 09 20:11:28 +0000 2015,RT @AnnaMedaris: Any dads out their who strugg...
2,586248551811932160,Thu Apr 09 19:25:21 +0000 2015,America's problem with diabetes in one map: ht...
3,586229697165586432,Thu Apr 09 18:10:26 +0000 2015,Think water &amp; fiber will cure your constip...
4,586215972731822080,Thu Apr 09 17:15:53 +0000 2015,"About to lose it? Here, try one of these offic..."


In [99]:
# Preprocessing
def clean_tweet(text):
  # Remove any word starting with @
  text = re.sub(r'@\w+', '', text)

  # Remove any hashtag symbols
  text = re.sub(r'#', '', text)

  # Remove any URL
  text = re.sub(r'http\S+', '', text)

  # Convert every word to lowercase
  text = text.lower()

  return text

df["clean"] = df["text"].apply(clean_tweet)

# Convert tweets into sets of words
df["word_set"] = df["clean"].apply(lambda x: set(x.split()))


In [100]:
# Compute Jaccard Distance
def jaccard_distance(a, b):
  if not a or not b:
    return 0
  return 1 - len(a & b) / len(a | b)


In [101]:
# Centroid Computing
def compute_centroids(cluster_word_sets):
  if len(cluster_word_sets) == 0:
    return None

  best_tweet = None
  best_score = float('inf')

  for i, t in enumerate(cluster_word_sets):
    dist_sum = sum(jaccard_distance(t, other) for other in cluster_word_sets)
    if dist_sum < best_score:
      best_score = dist_sum
      best_tweet = t

  return best_tweet

In [102]:
# Perform K-means clustering
def k_means_jaccard(tweets, K, max_iters=20):
  # Randomly pick K tweets as initial centroids
  centroids = random.sample(tweets, K)

  for iteration in range(max_iters):
    clusters = {i: [] for i in range(K)}

    # Assign each tweet to the closest centroid
    for tweet in tweets:
      distances = [jaccard_distance(tweet, c) for c in centroids]
      cluster_id = distances.index(min(distances))
      clusters[cluster_id].append(tweet)

    # Compute new centroids
    new_centroids = []
    for i in range(K):
      if len(clusters[i]) == 0:
        # If empty cluster, reinitialize randomly
        new_centroids.append(random.choice(tweets))
      else:
        new_centroids.append(compute_centroids(clusters[i]))

    # Check for convergence
    if new_centroids == centroids:
      break

    centroids = new_centroids

  return centroids, clusters

In [103]:
# SSE Computation
def compute_sse(centroids, clusters):
  sse = 0
  for i, c in clusters.items():
      centroid = centroids[i]
      for tweet in c:
          dist = jaccard_distance(tweet, centroid)
          sse += dist ** 2
  return sse

In [106]:
# Perform for multiple K and create table
K_values = [5, 10, 15, 20, 25, 30]
results = []

tweets = df["word_set"].tolist()

for K in K_values:
    print(f"Running K = {K} ...")
    centroids, clusters = k_means_jaccard(tweets, K)
    sse = compute_sse(centroids, clusters)

    cluster_sizes = {i: len(clusters[i]) for i in clusters}

    results.append({
        "K": K,
        "SSE": round(sse, 2),
        "Cluster Sizes": cluster_sizes
    })

Running K = 5 ...
Running K = 10 ...
Running K = 15 ...
Running K = 20 ...
Running K = 25 ...
Running K = 30 ...


In [108]:
# Display table
results_df = pd.DataFrame(results)

# 25 is around the best value of K
results_df

,K,SSE,Cluster Sizes
0,5,1126.60,"{0: 306, 1: 130, 2: 44, 3: 418, 4: 497}"
1,10,1094.46,"{0: 120, 1: 120, 2: 64, 3: 243, 4: 76, 5: 302,..."
2,15,1057.64,"{0: 129, 1: 55, 2: 21, 3: 102, 4: 46, 5: 155, ..."
3,20,1035.25,"{0: 19, 1: 49, 2: 29, 3: 160, 4: 102, 5: 39, 6..."
4,25,1010.97,"{0: 36, 1: 55, 2: 79, 3: 32, 4: 63, 5: 17, 6: ..."
5,30,1019.60,"{0: 22, 1: 47, 2: 30, 3: 63, 4: 27, 5: 24, 6: ..."
